In [97]:
import numpy as np
import pandas as pd
import random
from sklearn.cross_validation import train_test_split
from sklearn.datasets import load_iris
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

In [98]:
seed = 1
file = open("test.txt", "w")
# for a year of weather data
for x in range(0, 365):
    timpeve = random.randint(0, 1)
    impeve = str(timpeve)
    tdecibel = random.randint(60, 70)
    decibel = str(tdecibel)
    halfa = random.randint(0, 23)
    halfb = random.randint(0, 59)
    if halfa == 0:
        if halfb < 10:
            time = "000" + str(halfb)
        else:
            time = "00" + str(halfb)
    else:
        if halfa < 10:
            if halfb < 10:
                time = "0" + str(halfa) + "0" + str(halfb)
            else:
                time = "0" + str(halfa) + str(halfb)

        elif halfb < 10:
            if halfa < 10:
                time = "0" + str(halfa) + "0" + str(halfb)
            else:
                time = str(halfa) + "0" + str(halfb)
        else:
            time = str(halfa) + str(halfb)
    tfail = random.randint(0, 1)
    fail = str(tfail)
    tanomaly = random.randint(0, 1)
    anomaly = str(tanomaly)
    file.write(impeve + " " + decibel + " " + fail + " " + time + " " + anomaly + "\n")
    random.seed()
file.close()

In [99]:
data2 = pd.read_csv('888.csv');
data2 = data2.drop(['WT01', 'WT02','WT03','WT04','WT05','WT06','WT08','WT09', 'AWND', 'TAVG', 'STATION', 'STATION_NAME'], axis = 1)
# now we have ['date','prcp', 'snow', 'tmax', and 'tmin' as our features]
# but we want to specify LABC weather data, for this example
# so we make a copy of the original dataframe
# and splice it to only include the weather data from Los Angeles
# these are rows 216 - 581 in our .csv file 
labcWeather = data2[:]

labcWeather = labcWeather[216:582]

# we reset the index of every data value so that it can merge with our pseudo-random dataset
# 2016 was a leap year, so we have 366 entries

labcWeather = labcWeather.reset_index(drop=True)
print(labcWeather.head())

       DATE  PRCP    SNOW  TMAX  TMIN
0  20160103  0.00     0.0    62    45
1  20160104  0.00     0.0    67    56
2  20160105  1.42     0.0    59    52
3  20160106  1.18 -9999.0    57    47
4  20160107  0.12 -9999.0    58    48


In [100]:
# recall, labcWeather contains weather data for a year for LABC
# we can attach the values we created pseudo-randomly above to the features below
# and then combine the labcWeather data to include 


data = pd.read_csv('test.txt', sep=" ", header = None, names = ['ImpEvent', 'SigLvl', 'NearbyFail', 'Time', 'Anomaly'])
# we concatenate  
x = pd.merge(left=labcWeather, right=data,left_index = True, right_index = True)

# ['PRCP', 'SNOW', 'TMAX','TMIN', 'ImpEvent', 'SigLvl', 'NearbyFail','Time']
X = x[['DATE', 'PRCP', 'SNOW', 'TMAX', 'TMIN', 'ImpEvent', 'SigLvl', 'NearbyFail', 'Time']]
print(X)
y = x[['Anomaly']]
print(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=0)

logreg = LogisticRegression()
logreg.fit(X_train, y_train);

y_predict = logreg.predict(X_test)

score = accuracy_score(y_test, y_predict)
print(score)

         DATE  PRCP    SNOW  TMAX  TMIN  ImpEvent  SigLvl  NearbyFail  Time
0    20160103  0.00     0.0    62    45         0      63           1   506
1    20160104  0.00     0.0    67    56         0      70           1   126
2    20160105  1.42     0.0    59    52         1      65           0   220
3    20160106  1.18 -9999.0    57    47         1      68           1  1036
4    20160107  0.12 -9999.0    58    48         1      60           0  2034
5    20160108  0.00     0.0    61    47         0      67           0  1645
6    20160109  0.00 -9999.0    60    46         1      64           1  2357
7    20160110  0.01     0.0    60    51         1      69           1   706
8    20160111  0.00     0.0    62    49         0      61           1  2339
9    20160112  0.00     0.0    67    45         0      69           1   426
10   20160113  0.00     0.0    61    44         0      63           0   908
11   20160114  0.00     0.0    62    51         1      68           1    34
12   2016011

C:\Users\Dave Bravo-Ruelas\Anaconda3\lib\site-packages\sklearn\utils\validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
